# Свертки в Keras

Будем использовать Fashion MNIST

In [ ]:
!pip install -q tensorflow #==2.1

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf
print("We're using TF", tf.__version__)
import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
# print("We are using Keras", keras.__version__)
import pandas as pd

In [ ]:
from keras.models import Sequential

In [ ]:
# Проверяем что у нас работает GPU
tf.test.gpu_device_name()

In [ ]:
#GPU count and name
!nvidia-smi -L

# Загрузим данные

Классы:
```
0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot
```




In [ ]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
print(x_train.shape)
print(x_val.shape)
print (y_train[:10])
print (y_val[:10])

In [ ]:
x_train_image = np.tile(x_train[5, :, :].reshape((28, 28))[:, :, np.newaxis], (1, 1, 3))
print("Пример кусочка картинки:")
plt.imshow(x_train_image[15:20, 5:10], cmap="Greys")
plt.show()
print("Пример картинки:")
plt.imshow(x_train_image, cmap="Greys")
plt.show()
print("y_train [shape %s] 10 примеров:\n" % (str(y_train.shape)), y_train[:10])

In [ ]:
# one-hot encode для ответов
y_train_oh = keras.utils.to_categorical(y_train, 10)
y_val_oh = keras.utils.to_categorical(y_val, 10)

print(y_train_oh.shape, '\n')
print(y_train_oh[:5], '\n'*2, y_train[:5])

# Собираем модель

In [ ]:
K.clear_session()

model = Sequential()
model.add(L.Conv2D(1, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))

In [ ]:
model.summary()

In [ ]:
K.clear_session()

model = Sequential()
model.add(L.Conv2D(10, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))

In [ ]:
model.summary()

In [ ]:
K.clear_session()

model = Sequential()
model.add(L.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))
model.add(L.Conv2D(filters=32, kernel_size=3, strides=1, padding='same'))

In [ ]:
model.summary()

In [24]:
K.clear_session()

model.add(L.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))
model.add(L.MaxPool2D())
model.add(L.Conv2D(filters=64, kernel_size=3, strides=1, padding='same'))

ValueError: ignored

ValueError: ignored

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
Total params: 9568 (37.38 KB)
Trainable params: 9568 (37.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Будем как и в предыдущем модуле минимизировать кросс-энтропию.

In [ ]:
K.clear_session()

# в этот раз используем картинку как вход без вытягивания в вектор!

model = M.Sequential()
model.add(L.Conv2D(16, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))
model.add(L.MaxPool2D())
model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Conv2D(64, kernel_size=3, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Flatten())
model.add(L.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',  # минимизируем кросс-энтропию
    optimizer='adam',  # так будет быстрее, позже узнаем что это :)
    metrics=['accuracy']  # выводим процент правильных ответов
)

In [ ]:
# центрируем и нормируем, так сети будет проще учиться (позже узнаем почему :)
x_train_float = x_train.astype(np.float) / 255 - 0.5
x_val_float = x_val.astype(np.float) / 255 - 0.5

In [ ]:
# model.fit(
#     x_train_float[:, :, :, np.newaxis],
#     y_train_oh,
#     batch_size=32,
#     epochs=5,
#     validation_data=(x_val_float[:, :, :, np.newaxis], y_val_oh)
# )